In [ ]:
import yfinance as yf
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers
import pandas as pd
import random

import plotly
import cufflinks
plotly.__version__
from datetime import datetime, timedelta

## helper functions

"""
This grabs the start date given the end date and how
many days back you want to go
Input: string, int
Return: string
"""
def date_before_x_days(end_date, days_back):
  start_date = datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=days_back)
  return start_date.strftime('%Y-%m-%d')


"""
This deletes any stocks that doesn't have data from
our data set. Updates stocks left, data set, and the
stock count.
Input: list<string>, dataframe, int
Return: list<string>, dataframe, int
"""
def clean_data(stocks, dataset, stock_count):
  no_vals = []  ## stocks that don't have data
  for stock in stocks:
    d = dataset["Close"][stock].to_list()
    if any(np.isnan(i) for i in d):
      no_vals.append(stock)

  stock_count -= len(no_vals)  ## eliminating empty stock from count

  for stock in no_vals:
    stocks.remove(stock)  ## eliminating empty stock from STOCKS
    dataset = dataset.drop(columns=stock, level = 1)  ## dropping these stocks from our data

  return stocks, dataset, stock_count


"""
Given a dataset, we construct a dictionary representation
of our feature matrix to ensure the features are in the
same column. They are also labeled for simplicity
Outmost key is stock name
Inner key is feature name
Input: list<string>, dataframe, int
Return: dict<string, dict<string, int>>
"""
def make_feature_dict(stocks, dataset, window):
  X = {stock : {"const" : 1} for stock in stocks}  ## this will be our data

  ## Adj Close, Close, High, Low, Open, Volume (added here) (these are for a single day)
  for stock in stocks:
    X[stock]["Adj Close"] = dataset["Adj Close"][stock][-1]
    X[stock]["Close"] = dataset["Close"][stock][-1]
    X[stock]["High"] = dataset["High"][stock][-1]
    X[stock]["Low"] = dataset["Low"][stock][-1]
    X[stock]["Open"] = dataset["Open"][stock][-1]
    X[stock]["Volume"] = dataset["Volume"][stock][-1]

  ## SMA, Volatility, EMA, RSI (over last WINDOW_SIZE days) (these are more long-term information)
  for stock in stocks:
    g = dataset["Adj Close"][stock]  ## I used Adjusted Close for stability

    X[stock]["SMA"] = g.rolling(window=window).mean().iloc[-1]
    f = np.log(g / g.shift(1))
    X[stock]["Volatility"] = np.std(f[-window:])  ## this is how you calculate volatility
    X[stock]["EMA"] = g.ewm(span=window, adjust=False).mean().iloc[-1]

    delta = g.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))

    X[stock]["RSI"] = rsi.iloc[-1]
  return X


"""
Turns the Feature Matrix in Dictionary Form into an actual numpy matrix
"""
def dict_to_matrix(X):
  X_matrix = []
  for stock, features in X.items():
    temp = []

    for name, feature in features.items():
      temp.append(feature)

    X_matrix.append(temp)

  return np.array(X_matrix)


"""
Obtains the Y vector for the given stocks on the end_date
Input: string, int, list<string>
Return: list<int>
"""
def obtain_y(end_date, stocks, days_back=7):
  start_date = date_before_x_days(end_date, days_back)
  y_data = yf.download(stocks, start=start_date, end=end_date)["Adj Close"].iloc[-1]
  return np.array([y_data[stock] for stock in stocks])

In [ ]:
#######################################################################
## OBTAIN ALL COMPONENT STOCK FROM S&P 500 AND CREATE GLOBAL VARIABLES
#######################################################################
from sklearn.model_selection import train_test_split
URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_data = pd.read_html(URL)[0]

ALL_STOCKS = sp500_data['Symbol'].to_list()  ## grab random sample of this??
ALL_STOCKS = ALL_STOCKS[:50]
#ALL_STOCKS = random.sample(ALL_STOCKS, 100)
ALL_STOCK_COUNT = len(ALL_STOCKS)
#print(ALL_STOCK_COUNT)

# Replacing variable names (mandatory) # Cynthia added
TESTING_STOCKS = ALL_STOCKS
TRAINING_STOCKS = ALL_STOCKS

WINDOW_SIZE = 502  ## ALL THE DATA
GO_BACK = 50  ## DATA USED FOR ML


TRAINING_END_DATE = "2013-12-31"  ## training matrix data from this time period
TRAINING_START_DATE = date_before_x_days(TRAINING_END_DATE, WINDOW_SIZE)

TESTING_END_DATE = "2018-12-31"  ## testing matrix data from this time period
TESTING_START_DATE = date_before_x_days(TESTING_END_DATE, WINDOW_SIZE)


TRAINING_PREDICTION_DATE = "2018-12-31"
TESTING_PREDICTION_DATE = "2023-12-31"


In [ ]:
################################################################################
# OBTAIN DATA
################################################################################

## Training data for 2013. Used to predict in 2018
training_data = yf.download(ALL_STOCKS, start=TRAINING_START_DATE, end=TRAINING_END_DATE)

## Testing data for 2018. Used to predict in 2023
testing_data = yf.download(ALL_STOCKS, start=TESTING_START_DATE, end=TESTING_END_DATE)

[*********************100%%**********************]  50 of 50 completed
ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['ABNB', 'ANET']: Exception("%ticker%: Data doesn't exist for startDate = 1345089600, endDate = 1388466000")
[*********************100%%**********************]  50 of 50 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ABNB']: Exception("%ticker%: Data doesn't exist for startDate = 1502856000, endDate = 1546232400")


In [ ]:
################################################################################
# CLEANING DATA
################################################################################
ALL_STOCKS, training_data, ALL_STOCK_COUNT = clean_data(ALL_STOCKS,
                                                        training_data,
                                                        ALL_STOCK_COUNT)
print(ALL_STOCK_COUNT)
print(len(ALL_STOCKS))

ALL_STOCKS, testing_data, ALL_STOCK_COUNT = clean_data(ALL_STOCKS,
                                                        testing_data,
                                                        ALL_STOCK_COUNT)

print(ALL_STOCK_COUNT)
print(len(ALL_STOCKS))

ALL_STOCKS, training_data, ALL_STOCK_COUNT = clean_data(ALL_STOCKS,
                                                        training_data,
                                                        ALL_STOCK_COUNT)
print(ALL_STOCK_COUNT)
print(len(ALL_STOCKS))

46
46
46
46
46
46


In [ ]:
################################################################################
# POPULATE FEATURE MATRIX FOR TRAINING AND TESTING IN DICTIONARY FORM
################################################################################
training_X_dict = make_feature_dict(ALL_STOCKS,
                                    training_data,
                                    GO_BACK)

testing_X_dict = make_feature_dict(ALL_STOCKS,
                                   testing_data,
                                   GO_BACK)

In [ ]:
################################################################################
# TURN FEATURE MATRIX IN DICTIONARY FORM INTO A MATRIX
################################################################################
training_X = dict_to_matrix(training_X_dict)

testing_X = dict_to_matrix(testing_X_dict)

In [ ]:
################################################################################
# NORMALIZE THE FEATURE MATRIX FOR TRAINING AND TESTING
################################################################################
n = training_X.shape[1]
feature_norms = np.zeros(n)

for i in range(n):
  col = training_X[:, i]
  feature_norms[i] = np.linalg.norm(col)

training_X = training_X / feature_norms
testing_X = testing_X / feature_norms

In [ ]:
################################################################################
# OBTAIN Y VECTOR FOR TRAINING AND TESTING
################################################################################
training_y = obtain_y(TRAINING_PREDICTION_DATE, ALL_STOCKS)

testing_y = obtain_y(TESTING_PREDICTION_DATE, ALL_STOCKS)

[*********************100%%**********************]  46 of 46 completed
[*********************100%%**********************]  46 of 46 completed


In [ ]:
##########################################################################################################
# HELPER FUNCTIONS: OBTAIN OPTIMAL WEIGHTS USING RIDGE and HELPER FUNCTION FOR CALCULATING LOSS, OUR METHOD
##########################################################################################################
def ridge_obtain_thetas(training_X, training_y, lmbda = 2, iterations = 10000000, eps = 1e-3, alpha = 1e-5):
  cur_weights = np.zeros(training_X.shape[1])
  for _ in range(iterations):
    old_weights = cur_weights
    cur_weights = old_weights - alpha * (training_X.T.dot(training_X.dot(old_weights) - training_y) + lmbda * old_weights)
    if np.linalg.norm(cur_weights - old_weights) < eps:
      break
  else:
    print("increase iterations")

  theta = cur_weights
  return theta

# calculate loss
def calculate_loss(x, y, thetas, lmbda = 2):
  return (x.dot(thetas) - y).T.dot(x.dot(thetas) - y) + lmbda * thetas.T.dot(thetas)

In [ ]:
################################################################################
# EVALUATING LOSS ON TESTING DATA SET (OUR METHOD)
################################################################################
print("Loss on testing data set (ridge regression model)")
theta = ridge_obtain_thetas(training_X, training_y)

print(1 / len(testing_y) * calculate_loss(testing_X, testing_y, theta, lmbda = 2))
print(1 / len(testing_y) * calculate_loss(testing_X, testing_y, theta, lmbda = 0))

Loss on testing data set (ridge regression model)
11162.039302055455
10257.803461137175


In [ ]:
#####################################################################
# EVALUATING LOSS ON TESTING DATA SET (OPPOSING METHOD) (1-Year SMA)
#####################################################################
model_preds = []

NUM_YEARS = 5

for stock in TESTING_STOCKS:

  ## clip amount of data we need. Makes indexing easier
  g = testing_data["Adj Close"][stock][-252:]

  smas = np.zeros(12)
  adj_c = np.zeros(12)
  for i in range(12):
    g_temp = g[i * 21: (i+1) * 21]
    adj_c[i] = g_temp[0]
    smas[i] = g_temp.rolling(window=21).mean().iloc[-1]

  returns = (smas - adj_c) / adj_c
  ret = np.sum(returns)
  model_preds.append(((1 + ret) ** 5 ) * testing_data["Adj Close"][stock][-1])

print(model_preds)
model_preds = np.array(model_preds)

print("Loss on testing data set (1-year SMA, Method 1)")
print(1 / len(testing_y) * (model_preds - testing_y).T.dot(model_preds - testing_y))

[149.72341716927312, 40.98436365732366, 110.42369091173016, 210.3562918276093, 551.6506898855238, 238.74204367855484, 15.99744792126171, 48.4536788168675, 113.97234157578417, 206.38070755592554, 133.23640626842814, 89.06899297908865, 49.458113899996086, 530.640757702934, 30.569621773124325, 45.244787465945, 73.10469247046505, 77.05483832872517, 26.05461519526012, 200.6916021288991, 3.9820154481043692, 65.7042579059782, 7.913103141894963, 53.63360776703235, 130.3364004820943, 15.07546288512531, 169.7823818533614, 66.92532240370474, 37.79169022217633, 123.37024385984675, 267.1702539007744, 45.72507798744729, 112.6631362863609, 268.8133052166404, 225.66622251460063, 16.81326729229883, 64.45738334817327, 18.503274403005094, 59.49797740333652, 18.00031312485766, 67.46466926388636, 97.0236893572658, 76.89352198455839, 13.5308763254299, 83.32527095156487, 469.3224394822941]
Loss on testing data set (1-year SMA, Method 1)
8848.011336285965


In [ ]:
################################################################################
# Generating expected returns vector of TESTING using ML
################################################################################

proj = testing_X.dot(theta)
adj = testing_X[:, 1] * feature_norms[1]  ## un-normalizing the matrix
expected_returns_ridge = (proj - adj) / adj

actual = (testing_y - adj) / adj

print(np.power(1 / len(actual) * np.linalg.norm(actual - expected_returns_ridge) + 1, 1 / 5) - 1)
## We are off, on average, by 2.49% per year per stock when checking against testing
## Update: we are off, on average by 1.91% per year per stock when checking against testing

0.040512842367158575


In [ ]:
################################################################################
# (METHOD 1)
# Generating expected returns vector of testing stocks using 1-year SMA
################################################################################

expected_returns_SMA_1 = [] # Our goal is to populate this vector

NUM_YEARS = 5

for stock in ALL_STOCKS:

  ## clip amount of data we need. Makes indexing easier
  g = testing_data["Adj Close"][stock][-252:]

  smas = np.zeros(12)
  adj_c = np.zeros(12)
  for i in range(12):
    g_temp = g[i * 21: (i+1) * 21]
    adj_c[i] = g_temp[0]
    smas[i] = g_temp.rolling(window=21).mean().iloc[-1]

  returns = (smas - adj_c) / adj_c
  ret = np.sum(returns)

  expected_returns_SMA_1.append((1 + ret) ** NUM_YEARS - 1)

expected_returns_SMA_1 = np.array(expected_returns_SMA_1)
# print(expected_returns_SMA)

print(np.power(1 / len(actual) * np.linalg.norm(actual - expected_returns_SMA_1) + 1, 1 / 5) - 1)
## We are off, on average, by 2.66% per year per stock when checking against testing
## Update: We are off, on average, by 2.40% per year per stock when checking against testing

0.04155810490630607


In [ ]:
################################################################################
# (METHOD 2)
# Generating expected returns vector of testing stocks using 1-year SMA
################################################################################

import numpy as np

expected_returns_SMA_2 = [] # Our goal is to populate this vector

NUM_YEARS = 5
FACTOR =  1 / NUM_YEARS

for stock in ALL_STOCKS:

  g = testing_data["Adj Close"][stock][-252:]

  # Method 2: You could get the average adj_close_start from first 5 values
  adj_close_start = np.mean(g[0:5])

  sma = g.rolling(window=252).mean().iloc[-1]

  # Gets the return 251 (one year's market days) days before each end date
  ret = ( sma - adj_close_start ) / adj_close_start

  expected_returns_SMA_2.append((1 + ret) ** NUM_YEARS - 1)

expected_returns_SMA_2 = np.array(expected_returns_SMA_2)
# print(expected_returns_SMA)

print(np.power(1 / len(actual) * np.linalg.norm(actual - expected_returns_SMA_2) + 1, FACTOR) - 1)
## We are off, on average, by 2.70% per year per stock when checking against testing
## Update: We are off, on average, by 2.65% per year per stock when checking against testing

0.040657663092243945


In [ ]:
################################################################################
# Get Predicted Returns for a 50 day time span
################################################################################
predicted_prices = []

tr_data = training_data
te_data = testing_data

start_date = date_before_x_days(TRAINING_PREDICTION_DATE, 100)
training_y_data = yf.download(ALL_STOCKS, start=start_date, end=TRAINING_PREDICTION_DATE)["Adj Close"]

for i in range(GO_BACK):

  training_X_dict = make_feature_dict(ALL_STOCKS, tr_data, GO_BACK)

  testing_X_dict = make_feature_dict(ALL_STOCKS, te_data, GO_BACK)

  training_X = dict_to_matrix(training_X_dict)

  testing_X = dict_to_matrix(testing_X_dict)

  n = training_X.shape[1]
  feature_norms = np.zeros(n)

  for i in range(n):
    col = training_X[:, i]
    feature_norms[i] = np.linalg.norm(col)

  training_X = training_X / feature_norms
  testing_X = testing_X / feature_norms

  training_y_unordered = training_y_data.iloc[-i-1]
  training_y = np.array([training_y_unordered[stock] for stock in ALL_STOCKS])

  theta = ridge_obtain_thetas(training_X, training_y)

  predicted_prices.append(testing_X.dot(theta))

  tr_data = tr_data[:-1]
  te_data = te_data[:-1]

  if i + 1 % 5 == 0:
    print(i)

predicted_prices = predicted_prices[::-1]
predicted_prices = np.array(predicted_prices)
print(predicted_prices.shape) # 50 x 96?

[*********************100%%**********************]  46 of 46 completed


(50, 46)


In [ ]:
print(predicted_prices)
#predicted_prices = predicted_returns

[[204.95808076  65.31166681  86.67016644 ...  38.06394102 107.24397838
  170.94963569]
 [204.03326994  64.43693749  85.51740952 ...  38.10106872 107.90923778
  169.02958547]
 [202.60510801  63.45886869  84.52495229 ...  38.07671814 108.70500784
  166.11722563]
 ...
 [184.97354491  62.06019972  84.86350793 ...  38.79528965 102.49351143
  150.68010669]
 [188.10034361  62.33691935  86.15321257 ...  38.84876891 103.17555304
  152.23965163]
 [189.69292372  62.36782274  86.90240381 ...  39.01959176 103.95312087
  153.39396309]]


In [ ]:
################################################################################
# Calculate HISTORICAL Covariance Matrix
################################################################################
# Helper function
GO_BACK = 50  ## DATA USED FOR ML

def generate_daily_return_vec(closing_prices_vec):
  daily_returns_vec = []
  for i in range(1, len(closing_prices_vec)):
    daily_return = ((closing_prices_vec[i] - closing_prices_vec[i-1]) / closing_prices_vec[i-1]) * 100
    daily_returns_vec.append(daily_return)
  return daily_returns_vec

# Helper function to calculate the covariance between two assets
def calculate_covariance_hist(asset1, asset2):
  closing_prices_asset1 = testing_data["Adj Close"][asset1][-GO_BACK:]
  closing_prices_asset2 = testing_data["Adj Close"][asset2][-GO_BACK:]
  daily_returns_asset1, daily_returns_asset2 = generate_daily_return_vec(closing_prices_asset1), generate_daily_return_vec(closing_prices_asset2)
  sample_means_1, sample_means_2 = np.mean(daily_returns_asset1), np.mean(daily_returns_asset2)
  # Subtract the sample means from each element of the respective daily returns vectors
  deviation_1, deviation_2 = daily_returns_asset1 - sample_means_1, daily_returns_asset2 - sample_means_2
  # Next step
  un_normalized_covariance = np.sum(deviation_1 * deviation_2) # element-wise multiplication
  covariance = (1 / (GO_BACK - 1)) * un_normalized_covariance # Check that N - 1 is correct
  return covariance

# Function to calculate covariance matrix given a vector of stocks
def calculate_covariance_matrix_hist(stocks): #param: vector of stocks
  num_stocks = len(stocks)
  covariance_matrix = np.zeros((num_stocks, num_stocks))
  for i in range(num_stocks):
    asset1 = stocks[i]
    for j in range(num_stocks):
      asset2 = stocks[j]
      covariance_matrix[i, j] = calculate_covariance_hist(asset1, asset2)
  return covariance_matrix

covariance_matrix_historical = calculate_covariance_matrix_hist(TESTING_STOCKS)
print(covariance_matrix_historical)

[[ 3.49958807  2.04429075  2.76426011 ...  1.83309898  0.84261964
   4.33507654]
 [ 2.04429075  5.13812004  2.04610422 ...  1.34507494 -0.06389754
   4.1556187 ]
 [ 2.76426011  2.04610422  3.41017524 ...  1.78854287  0.51497066
   4.37688708]
 ...
 [ 1.83309898  1.34507494  1.78854287 ...  3.61313956  0.77917841
   3.13450027]
 [ 0.84261964 -0.06389754  0.51497066 ...  0.77917841  2.24102151
   0.05459386]
 [ 4.33507654  4.1556187   4.37688708 ...  3.13450027  0.05459386
  11.07585884]]


In [ ]:
################################################################################
# Calculate PROJECTED Covariance Matrix
################################################################################
# ret_vec_1 = predicted_returns[asset1]
# ret_vec_2 = predicted_returns[asset2]

# Get returns vector from predicted returns (030724)
def get_prices_vector(index, predicted_returns):
  ret_vec = []
  for row in predicted_returns:
    val = row[index]
    ret_vec.append(val)
  return ret_vec

# Inputs are predicted prices vectors of 50 days
def calculate_covariance_pred(daily_prices_asset1, daily_prices_asset2):
  N = len(daily_prices_asset1)
  daily_returns_asset1, daily_returns_asset2 = generate_daily_return_vec(daily_prices_asset1), generate_daily_return_vec(daily_prices_asset2)
  sample_means_1, sample_means_2 = np.mean(daily_returns_asset1), np.mean(daily_returns_asset2)
  # Subtract the sample means from each element of the respective daily returns vectors
  deviation_1, deviation_2 = daily_returns_asset1 - sample_means_1, daily_returns_asset2 - sample_means_2
  # Next step
  un_normalized_covariance = np.sum(deviation_1 * deviation_2) # element-wise multiplication
  covariance = (1 / (N - 1)) * un_normalized_covariance # Check that N - 1 is correct
  return covariance

# Getting predicted covariance matrix
def calculate_covariance_matrix_pred(stocks): #param: vector of stocks (TESTING_STOCKS)
  num_stocks = len(stocks)
  covariance_matrix = np.zeros((num_stocks, num_stocks))
  for i in range(num_stocks):
    asset1 = stocks[i]
    for j in range(num_stocks):
      asset2 = stocks[j]
      # New new stuff (031424)
      ret_vec_1 = get_prices_vector(i, predicted_prices)
      ret_vec_2 = get_prices_vector(j, predicted_prices)
      # Helper function above: calculate_covariance_pred
      covariance_matrix[i, j] = calculate_covariance_pred(ret_vec_1, ret_vec_2)
  return covariance_matrix

covariance_matrix_pred = calculate_covariance_matrix_pred(TESTING_STOCKS)
print(covariance_matrix_pred)

[[1.38366256 0.72826517 1.14112161 ... 0.4838617  0.51981948 1.20447038]
 [0.72826517 1.55246882 0.74385607 ... 0.19392819 0.09879106 1.07162827]
 [1.14112161 0.74385607 1.43033425 ... 0.68568201 0.32407817 1.27502712]
 ...
 [0.4838617  0.19392819 0.68568201 ... 3.69281305 0.57842185 0.20027396]
 [0.51981948 0.09879106 0.32407817 ... 0.57842185 0.80956287 0.14574575]
 [1.20447038 1.07162827 1.27502712 ... 0.20027396 0.14574575 3.0329736 ]]


In [ ]:
#################################################################################
# MARKOWITZ MODELS FUNCTIONS
# BUNDLED UP SIMPLE AND COMPLEX MARKOWITZ MODELS INTO TWO HELPER FUNCTIONS
#################################################################################
import numpy as np
from scipy.optimize import minimize

# Simple model
def calculate_markowitz_weights_simple(sigma_squared, expected_returns, covariance_matrix):

  def objective(weights, expected_returns):
    return -np.dot(weights, expected_returns)

  # Weights must be between 0 and 1
  bounds = tuple((0, 1) for _ in range(len(expected_returns)))

  # Initial guess for weights
  initial_guess = np.ones(len(expected_returns)) / len(expected_returns) # [1/5, 1/5, etc.]

  constraint = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

  result = minimize(objective, initial_guess, args=(expected_returns,), method='SLSQP', bounds=bounds, constraints=constraint)
  optimal_weights_complex = result.x
  return optimal_weights_complex

# Complex model
def calculate_markowitz_weights_complex(sigma_squared, expected_returns, covariance_matrix):

  def objective(weights, expected_returns):
    return -np.dot(weights, expected_returns)

  # Set bounds for individual weights (0 <= weight <= 1)
  bounds = tuple((0, 1) for _ in range(len(expected_returns)))

  # Initial guess for weights
  initial_guess = np.ones(len(expected_returns)) / len(expected_returns) # [1/5, 1/5, etc.]

  # Constraints
  constraint_1 = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
  constraint_2 = {'type': 'ineq', 'fun': lambda weights: sigma_squared - np.dot(weights.T, np.dot(covariance_matrix, weights))}
  constraints = [constraint_1, constraint_2]

  result = minimize(objective, initial_guess, args=(expected_returns,), method='SLSQP', bounds=bounds, constraints=constraints)
  optimal_weights_complex = result.x
  return optimal_weights_complex

In [ ]:
######################################################################
# Running each of the 4 combinations for BOTH simple AND complex (31424)
######################################################################
all_weights_simple = []
all_weights_complex = []
covariance_matrix_historical = calculate_covariance_matrix_hist(TESTING_STOCKS)
covariance_matrix_pred = calculate_covariance_matrix_pred(TESTING_STOCKS)

expected_returns = [expected_returns_ridge, expected_returns_SMA_1] # changed from expected_returns_SMA to expected_returns_SMA_1
covariances = [covariance_matrix_historical, covariance_matrix_pred]

for i in range(len(expected_returns)):
  exp_vec = expected_returns[i]
  for j in range(len(covariances)):
    cov_matrix = covariances[j]
    weights_simple = calculate_markowitz_weights_simple(0.15, exp_vec, cov_matrix)
    weights_complex = calculate_markowitz_weights_complex(0.15, exp_vec, cov_matrix)
    all_weights_simple.append(weights_simple)
    all_weights_complex.append(weights_complex)

print(all_weights_simple)
print(all_weights_complex) # So far, this looks correct

[array([0.00000000e+00, 0.00000000e+00, 2.14230131e-15, 0.00000000e+00,
       2.96026405e-15, 3.16413562e-15, 2.15966173e-15, 1.20940127e-15,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.26342378e-15, 0.00000000e+00, 0.00000000e+00, 5.98914187e-16,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.00000000e+00, 3.07591236e-15, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 4.27978743e-16, 0.00000000e+00,
       3.87884285e-15, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 5.73325086e-16, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 6.61094545e-16]), array([0.00000000e+00, 0.00000000e+00, 2.14230131e-15, 0.00000000e+00,
       2.96026405e-15, 3.16413562e-15, 2.15966173e-15, 1.20940127e-15,
       0.00000000e+00, 0.00000000e+

In [ ]:
##############################################################################################################
# Evaluating the performance of each of the the Markowitz weights for BOTH the complex and simple models (31424)
# After obtaining weights, check the return of the model over the past 5 years
# if you invested at time t = 2018.
##############################################################################################################
END_DATE = "2018-12-28"
PREDICTION_END_DATE = "2023-12-28"
# We will assume we have a weights vector for each stock in the TESTING_STOCKS
#optimal_weights = # vector, we got this from running Markowitz model
optimal_weights_simple = all_weights_simple
optimal_weights_complex = all_weights_complex

def get_actual_returns(TESTING_STOCKS):
  actual_returns_testing = []
  for i in range(len(TESTING_STOCKS)):
    stock = TESTING_STOCKS[i]
    # Grab beginning value (Dec 28 2018)
    beginning_val = testing_data["Adj Close"][stock][END_DATE]
    ending_val = testing_y[i]
    ret = ((ending_val - beginning_val) / beginning_val) * 100
    actual_returns_testing.append(ret)
  return actual_returns_testing

def calculate_portfolio_return(weight_vec):
  actual_returns = get_actual_returns(TESTING_STOCKS)
  portfolio_return = weight_vec.dot(actual_returns)
  return portfolio_return

print("Markowitz model with 1 constraint")
print("Order: ridge-historical, ridge-pred, sma-historical, sma-pred")
for weight_vec in optimal_weights_simple:
  portfolio_return = calculate_portfolio_return(weight_vec)
  print(portfolio_return)

print("Markowitz model with 2 constraints")
print("Order: ridge-historical, ridge-pred, sma-historical, sma-pred")
for weight_vec in optimal_weights_complex:
  portfolio_return = calculate_portfolio_return(weight_vec)
  print(portfolio_return)

Markowitz model with 1 constraint
Order: ridge-historical, ridge-pred, sma-historical, sma-pred
30.089232846481444
30.089232846481444
727.2166452666834
727.2166452666834
Markowitz model with 2 constraints
Order: ridge-historical, ridge-pred, sma-historical, sma-pred
49.80890414356125
44.74011310674608
49.80871109742626
44.741334854290535


In [ ]:
###################################################################
# Nolawi's Thetas and Covariance Matrices
###################################################################
thetas = []
X_matr = []
cov_matr = []
all_ret = []

data = yf.download(ALL_STOCKS, start="2018-8-20", end="2023-11-30")

y_data = yf.download(ALL_STOCKS, start="2018-8-20", end="2023-11-30")["Adj Close"]

for i in range(60):

  testing_X_dict = make_feature_dict(ALL_STOCKS, data, GO_BACK)

  ret = []
  for stock in ALL_STOCKS:
    ret.append((data["Adj Close"][stock].iloc[-1] - data["Adj Close"][stock].iloc[-21]) / data["Adj Close"][stock].iloc[-21])
  all_ret.append(ret)

  daily_ret_unordered = data[-22:]["Adj Close"].pct_change()[-21:]

  daily_ret = np.array([daily_ret_unordered[stock] for stock in ALL_STOCKS])

  covariance_matrix = np.cov(daily_ret)

  cov_matr.append(covariance_matrix)

  data = data[:-21]

  training_X_dict = make_feature_dict(ALL_STOCKS, data, GO_BACK)

  testing_X = dict_to_matrix(testing_X_dict)

  training_X = dict_to_matrix(training_X_dict)

  n = training_X.shape[1]
  feature_norms = np.zeros(n)

  for i in range(n):
    col = training_X[:, i]
    feature_norms[i] = np.linalg.norm(col)

  training_X = training_X / feature_norms

  testing_X = testing_X / feature_norms

  X_matr.append(testing_X)

  training_y_unordered = y_data.iloc[-1]
  training_y = np.array([training_y_unordered[stock] for stock in ALL_STOCKS])

  y_data = y_data[:-21]

  thetas.append(ridge_obtain_thetas(training_X, training_y))


thetas = thetas[::-1]
X_matr = X_matr[::-1]
cov_matr = cov_matr[::-1]
all_ret = all_ret[::-1]

print(len(thetas))
print(len(X_matr))
print(len(cov_matr))

[*********************100%%**********************]  46 of 46 completed
[*********************100%%**********************]  46 of 46 completed


60
60
60


In [ ]:
print(cov_matr[0].shape)

(46, 46)


In [ ]:
################################################################################
# NOLAWI'S CELL FOR Reinforcement Learning
################################################################################
import numpy as np
import tensorflow as tf

stocks = ALL_STOCKS

sigma_squared = 0.15

def calculate_reward(weights, feature_matrix, theta, covariance_matrix):
  weights = weights / np.linalg.norm(weights)
  if weights.T.dot(covariance_matrix.dot(weights)) > sigma_squared:
    return -1000
  return weights.T.dot(feature_matrix.dot(theta))


def get_reward(actions, weights, feature_matrix, theta, covariance_matrix):
  for i in range(len(actions)):
    if actions[i] == 0:
      weights[i] = np.random.rand()
  return calculate_reward(weights, feature_matrix, theta, covariance_matrix)


# Define the policy network
class PolicyNetwork(tf.keras.Model):
    def __init__(self):
        super(PolicyNetwork, self).__init__()
        self.dense = tf.keras.layers.Dense(units=2, activation='softmax')

    def call(self, state):
        return self.dense(state)

# Initialize policy network and optimizer
policy_network = PolicyNetwork()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# Training loop
num_episodes = 200


best_weights = []
# for date in last_days_of_months:
for b in range(len(thetas)):

    weights = np.random.uniform(0,1,len(stocks))
    weights /= np.sum(weights)

    max_reward = -2000
    max_weights = weights
    for episode in range(num_episodes):
      # Initialize lists to store episode data
      episode_weights = []
      episode_actions = []
      episode_rewards = []
      episode_matrices = []

      action_probs = policy_network(X_matr[b])
      # print(X_matr[b])
      # print(action_probs)
      actions = []

      for i in range(len(stocks)):
        actions.append(np.random.choice(len(action_probs[i]), p=action_probs.numpy()[i]))

      actions = np.array(actions)

      # Take action and observe reward
      this_reward = get_reward(actions, weights, X_matr[b], thetas[b], cov_matr[b])
      if this_reward > max_reward:
        max_weights = weights
        max_reward = this_reward

        # Store state, action, and reward
      episode_weights.append(weights)
      episode_actions.append(actions)
      episode_rewards.append(this_reward)
      episode_matrices.append(X_matr[b])

      episode_rewards = np.array(episode_rewards)
        # Compute loss and update policy
      with tf.GradientTape() as tape:
            action_probs = policy_network(np.vstack(episode_matrices))

            action_masks = tf.one_hot(episode_actions, len(action_probs[0]))

            log_probs = tf.reduce_sum(action_masks * tf.math.log(action_probs), axis=1)
            loss = -tf.reduce_mean(log_probs * episode_rewards)

      gradients = tape.gradient(loss, policy_network.trainable_variables)
      optimizer.apply_gradients(zip(gradients, policy_network.trainable_variables))

    # Print episode information

      print(f"Iteration: {b} Episode: {episode}, Max Reward: {max_reward}")
      # print(len(episode_actions[0]))
    max_weights /= np.sum(max_weights)
    best_weights.append(max_weights)
print(best_weights)

Streaming output truncated to the last 5000 lines.
Iteration: 37 Episode: 163, Max Reward: 756.1131165309026
Iteration: 37 Episode: 164, Max Reward: 756.1131165309026
Iteration: 37 Episode: 165, Max Reward: 756.1131165309026
Iteration: 37 Episode: 166, Max Reward: 756.1131165309026
Iteration: 37 Episode: 167, Max Reward: 756.1131165309026
Iteration: 37 Episode: 168, Max Reward: 756.1131165309026
Iteration: 37 Episode: 169, Max Reward: 756.1131165309026
Iteration: 37 Episode: 170, Max Reward: 756.1131165309026
Iteration: 37 Episode: 171, Max Reward: 756.1131165309026
Iteration: 37 Episode: 172, Max Reward: 756.1131165309026
Iteration: 37 Episode: 173, Max Reward: 756.1131165309026
Iteration: 37 Episode: 174, Max Reward: 756.1131165309026
Iteration: 37 Episode: 175, Max Reward: 756.1131165309026
Iteration: 37 Episode: 176, Max Reward: 756.1131165309026
Iteration: 37 Episode: 177, Max Reward: 756.1131165309026
Iteration: 37 Episode: 178, Max Reward: 756.1131165309026
Iteration: 37 Episode

In [ ]:
best_weights = np.array(best_weights)
all_ret = np.array(all_ret)
portfolio_returns_RL = sum([w.T.dot(r) for w, r in zip(best_weights, all_ret)])
print(portfolio_returns_RL)

0.7139087028396547
